In [1]:
import numpy as np
import names
import random
from collections import OrderedDict, deque, defaultdict
import alex_email as amail

In [2]:
class CustomError(Exception):
    """Base class for exceptions in this module."""
    pass

In [3]:
class CardNumberError(CustomError):
    def __init__(self, message, number):
        self.message = message
        self.number = number
    def __str__(self):
        return f"The numer {self.number} is {self.message} than the allowed card number range: 1~52."
class CardNumberDisaster(CustomError):
    def __init__(self,card):
        self.card = card
    def __str__(self):
        return f"The card numer {self.card.name()} is illegal."

In [4]:
class Card:
    suit_map = {0: 'Heart',
                1: 'Diamond',
                2: 'Spade',
                3: 'Club'}
    def __init__(self, number, card_code=None):
        if number > 52:
            raise CardNumberError('larger', number)
        elif number < 1:
            raise CardNumberError('smaller', number)
        else:
            self.number = (number-1)%13+1
            self.suit = Card.suit_map[(number-1)//13]
            self.card_id = number
            self.left = None
            self.right = None
            self.previous = None
            self.available = False
            self.onboard = False
            self.suppressed = False
            self.player_id = None
            self.name = self.name()
            self.card_code = number if card_code is None else card_code
    def __str__(self):
        return f"This card is {self.suit} {self.number}. The uid of this card is {self.card_id}."
    
    def name(self):
        return f"{self.suit} {self.number}"
    
    def issue(self):
        self.onboard = True
        self.available = False
        if self.left is not None:
            self.left.available = True
        if self.right is not None:
            self.right.available = True
            
    def suppress(self):
        self.suppressed = True
        self.available = False    
        

In [5]:
class Player:
    identity = 0
    def __init__(self, name=None, email=None):
        Player.identity += 1
        self.name = name or names.get_first_name()
        self.email = email
        self.player_id = Player.identity
        self.hand_card = None
        self.available = []
        self.suppressed = {}
        self.score = 0
        self.in_game = True
        self.decoder = {}
    
    def send(self, text, round_count, game_id):
        if self.email is None:
            print(text)
        else:
            try:
                amail.send_mail([self.email], text, subject=f"Suppress Seven Game {game_id} Round {round_count+1}")
            except Exception as e:
                print(text)
    
    def get_hand_cards(self, hand_card):
        for card in hand_card:
            card.player_id = self.player_id
            self.decoder[card.card_code] = card.card_id
        self.hand_card = OrderedDict(sorted([(card.card_id, card) for card in hand_card]))
    
    def _check_available(self):
        self.available = []
        for card_id, card in self.hand_card.items():
            if card.available:
                self.available.append(card_id)
                
    def show_available(self):
        send_text = "You current available cards are:\n"
        for card_id in self.available:
            send_text += (f"{self.hand_card[card_id].name} is available, " 
                          + f"use card code {self.hand_card[card_id].card_code} if you want to use.\n")
        return send_text        
            
    def show_hand_cards(self):
        send_text = "You currently have:\n"
        suppress_text = "\nYou have to suppress: \n"
        hand_cards = defaultdict(list)
        suppressed_cards = defaultdict(list)
        for card_id in self.hand_card.keys():
            hand_cards[self.hand_card[card_id].suit].append(str(self.hand_card[card_id].number))
            suppress_text += (f"{self.hand_card[card_id].name} is in your hand, " 
                              + f"use card code {self.hand_card[card_id].card_code} if you want to suppress. \n")
        for card_id in self.suppressed.keys():
            suppressed_cards[self.suppressed[card_id].suit].append(str(self.suppressed[card_id].number))
        for suit in hand_cards.keys():
            send_text += f"{suit} {', '.join(hand_cards[suit])}.\n"
        send_text += '\nYou have suppressed: \n'
        for suit in suppressed_cards.keys():
            send_text += f"{suit} {', '.join(suppressed_cards[suit])}.\n"    
        if len(self.available) == 0:
            send_text += suppress_text
        return send_text 
    
    def issue(self, card_id):
        self._check_available()
        if len(self.available) == 0:
            print("There is no card available, please suppress one instead.")
            return False
        elif card_id in self.available:
            issued_card = self.hand_card.pop(card_id)
            issued_card.issue()
            return True
        elif card_id in self.hand_card.keys():
            print("This card is not available, please try again.")
            return False
        elif card_id < 1 or card_id > 52:
            print("Not a valid card id, please try again.")
            return False
        else:
            print('Not in your hand, please try again.')
            return False
    
    def suppress(self, card_id):
        self._check_available()
        if len(self.available) > 0:
            print("You have at least one card available, you need to issue one card.")
            return False
        elif card_id in self.available:
            print("This card is available, you need to issue one card.")
            return False
        elif card_id not in self.hand_card.keys():
            print("You don't have this card, please choose another one.")
            return False
        elif card_id in list(self.hand_card.keys()):
            suppressed_card = self.hand_card.pop(card_id)
            suppressed_card.suppress()
            self.suppressed[card_id] = suppressed_card
            self.score += suppressed_card.number
            return True
        else:
            print("Try again!")
            return False
        
    def action(self, round_count, game_id):
        self._check_available()
        if len(self.available) > 0:
            send_text = self.show_hand_cards() + '\n' + self.show_available()
            self.send(send_text, round_count, game_id)
            card_code = input("Please input the card code you want to issue: ")
            try:
                card_code = int(card_code)
                card_id = self.decoder[card_code]
            except:
                card_id = 0
            return self.issue(card_id)
        elif len(self.hand_card) > 0:
            send_text = self.show_hand_cards()
            self.send(send_text, round_count, game_id)
            card_code = input("Please input the card code you want to suppress: ")
            try:
                card_code = int(card_code)
                card_id = self.decoder[card_code]
            except:
                card_id = 0
            return self.suppress(card_id)
        else:
            print("You have no card left, pass.")
            return True    
        

In [6]:
class Game:
    identity = 0
    def __init__(self, random_seed=13):
        Game.identity += 1
        self.game_id = Game.identity
        self.card_map = OrderedDict()
        self.player_map = OrderedDict()
        self.card_pool = []
        self.players = deque([])
        self.onboard_cards = []
        self.min_score = 9999
        self.winner = None
        pool = list(range(1,53))
        random.seed(random_seed)
        random.shuffle(pool)
        self.encoder = {}
        for i, v in enumerate(pool):
            self.encoder[i+1] = v
        
    def get_cards(self):
        previous_card = None
        for i in range(1,53):
            this_card = Card(i, self.encoder[i])
            if this_card.number < 7:
                this_card.left = previous_card
                if previous_card is not None:
                    previous_card.previous = this_card
            elif this_card.number > 7:
                this_card.previous = previous_card
                if previous_card is not None:
                    previous_card.right = this_card
            elif this_card.number == 7:
                this_card.left = previous_card
                previous_card.previous = this_card
                this_card.available = True

            else:
                raise CardNumberDisaster(this_card)
            if this_card.number == 13:
                previous_card = None
            else:
                previous_card = this_card
            self.card_map[this_card.card_id] = this_card
        self.card_pool = list(self.card_map.values())
        random.shuffle(self.card_pool)
    
    def prepare(self):
        for i in range(4):
            this_name = input("Please input your name: ")
            this_name = None if this_name == '' else this_name
            this_email = input("Please input your email: ")
            this_email = None if this_email == '' else this_email
            this_player = Player(this_name, this_email)
            print(f"Welcome {this_player.name}, have fun!")
            this_player.get_hand_cards(self.card_pool[i*13:(i+1)*13])
            self.player_map[this_player.player_id] = this_player
            self.players.append(this_player)
            
        init_player = self.player_map[self.card_map[33].player_id]
        first_player_index = self.players.index(init_player)
        self.players.rotate(-1*first_player_index)
    def check_onboard(self, show=True):
        self.onboard_cards = []
        for card_id in self.card_map.keys():
            if self.card_map[card_id].onboard:
                self.onboard_cards.append(self.card_map[card_id])
        if show:
            current_onboard = defaultdict(list)
            for card in self.onboard_cards:
                current_onboard[card.suit].append(str(card.number))
            print("\nCurrent onboard cards are:")
            print("************************************************")
            for suit in current_onboard.keys():
                print(f"{suit} {', '.join(current_onboard[suit])}.")
            print("************************************************")

        
    def one_round(self, round_count, game_id):
        for player in self.players:
            self.check_onboard(True)
            print(f"\nIt is {player.name}'s turn.\n")
            while True:
                action_result = player.action(round_count, game_id)
                if action_result:
                    break
    
    def on_going(self):
        round_count = 0
        while round_count < 13:
            self.one_round(round_count, self.game_id)
            round_count += 1
    
        for player in self.players:
            print(f"{player.name} has {player.score} points.")
            if player.score < self.min_score:
                self.min_score = player.score
                self.winner = player
        print(f"{self.winner.name} wins!")

In [7]:
test_game = Game()

In [8]:
test_game.get_cards()

In [9]:
test_game.prepare()

Please input your name: 
Please input your email: 
Welcome Ronald, have fun!
Please input your name: 
Please input your email: 
Welcome Ruth, have fun!
Please input your name: 
Please input your email: 
Welcome William, have fun!
Please input your name: 
Please input your email: 
Welcome Alton, have fun!


In [10]:
test_game.on_going()


Current onboard cards are:
************************************************
************************************************

It is Ruth's turn.

You currently have:
Heart 1, 2, 6, 7, 8.
Diamond 2.
Spade 3, 5, 7, 12.
Club 2, 8, 9.

You current available cards are:
Heart 7 is available, use card code 21 if you want to use.
Spade 7 is available, use card code 40 if you want to use.

Please input the card code you want to issue: 40

Current onboard cards are:
************************************************
Spade 7.
************************************************

It is William's turn.

You currently have:
Heart 3, 5, 9.
Diamond 1, 3, 8, 13.
Spade 2, 6, 11.
Club 5, 10, 13.

You current available cards are:
Spade 6 is available, use card code 1 if you want to use.

Please input the card code you want to issue: 1

Current onboard cards are:
************************************************
Spade 6, 7.
************************************************

It is Alton's turn.

You currently have

Please input the card code you want to issue: 49

Current onboard cards are:
************************************************
Heart 4, 5, 6, 7.
Diamond 5, 6, 7, 8, 9.
Spade 5, 6, 7, 8, 9.
Club 7.
************************************************

It is Ruth's turn.

You currently have:
Heart 1, 2, 8.
Diamond 2.
Spade 3, 12.
Club 2, 8, 9.

You current available cards are:
Heart 8 is available, use card code 22 if you want to use.
Club 8 is available, use card code 42 if you want to use.

Please input the card code you want to issue: 42

Current onboard cards are:
************************************************
Heart 4, 5, 6, 7.
Diamond 5, 6, 7, 8, 9.
Spade 5, 6, 7, 8, 9.
Club 7, 8.
************************************************

It is William's turn.

You currently have:
Heart 3.
Diamond 1, 3, 13.
Spade 2, 11.
Club 5, 10, 13.

You current available cards are:
Heart 3 is available, use card code 52 if you want to use.

Please input the card code you want to issue: 52

Current onboard c

Please input the card code you want to issue: 39

Current onboard cards are:
************************************************
Heart 1, 2, 3, 4, 5, 6, 7.
Diamond 4, 5, 6, 7, 8, 9, 10.
Spade 3, 4, 5, 6, 7, 8, 9, 10, 11.
Club 7, 8.
************************************************

It is William's turn.

You currently have:
Diamond 1, 3, 13.
Spade 2.
Club 10, 13.

You current available cards are:
Diamond 3 is available, use card code 20 if you want to use.
Spade 2 is available, use card code 3 if you want to use.

Please input the card code you want to issue: 3

Current onboard cards are:
************************************************
Heart 1, 2, 3, 4, 5, 6, 7.
Diamond 4, 5, 6, 7, 8, 9, 10.
Spade 2, 3, 4, 5, 6, 7, 8, 9, 10, 11.
Club 7, 8.
************************************************

It is Alton's turn.

You currently have:
Heart 12, 13.
Diamond 11.
Spade 13.
Club 6, 12.

You current available cards are:
Diamond 11 is available, use card code 45 if you want to use.
Club 6 is availabl

Please input the card code you want to issue: 12

Current onboard cards are:
************************************************
Heart 1, 2, 3, 4, 5, 6, 7, 8.
Diamond 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13.
Spade 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13.
Club 6, 7, 8, 9.
************************************************

It is William's turn.

You currently have:
Club 10, 13.

You current available cards are:
Club 10 is available, use card code 50 if you want to use.

Please input the card code you want to issue: 50

Current onboard cards are:
************************************************
Heart 1, 2, 3, 4, 5, 6, 7, 8.
Diamond 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13.
Spade 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13.
Club 6, 7, 8, 9, 10.
************************************************

It is Alton's turn.

You currently have:
Heart 13.
Club 12.

You have to suppress: 
Heart 13 is in your hand, use card code 11 if you want to suppress. 
Club 12 is in your hand, use card code 19 if yo

In [10]:
a = list(range(1,53))
random.seed(13)
random.shuffle(a)
d = {}
for i, v in enumerate(a):
    d[i] = v

In [13]:
e = {y:x for x,y in d.items()}

In [14]:
e

{32: 0,
 34: 1,
 52: 2,
 49: 3,
 36: 4,
 4: 5,
 21: 6,
 22: 7,
 18: 8,
 38: 9,
 13: 10,
 6: 11,
 11: 12,
 7: 13,
 24: 14,
 20: 15,
 23: 16,
 16: 17,
 8: 18,
 41: 19,
 25: 20,
 29: 21,
 37: 22,
 45: 23,
 31: 24,
 30: 25,
 26: 26,
 3: 27,
 39: 28,
 27: 29,
 33: 30,
 1: 31,
 40: 32,
 28: 33,
 2: 34,
 51: 35,
 14: 36,
 35: 37,
 5: 38,
 9: 39,
 48: 40,
 47: 41,
 46: 42,
 10: 43,
 43: 44,
 15: 45,
 42: 46,
 12: 47,
 50: 48,
 44: 49,
 19: 50,
 17: 51}